# Prediction Input Parameters
1. Poolreturntemp (Done)
1. OATemp (Done)
1. OAHum **
1. uv index (Done)
1. HourOfDay (Need to extract)
1. DayOfYear (Need to extract)
1. Flow (PumpPower) **


# Other Inputs
1. "Efficiency" (Done)

# Captured Trainable Output Parameters
1. Predicted Temp Rise (poolheatedtemp, done)
1. Predicted PoolReturnTemp (t+1h)
1. Predicted PoolReturnTemp (t+3h)
1. Predicted HeatingPowerDemand (Done)

# Historical Data Rows
1. t
1. t+5min
1. t+10min

# Historical Data
## Inputs
1. OATemp
1. OAHum
1. CloudCover/SolarLoad/UV Index
1. PoolReturnTemp
1. HourOfDay
1. DayOfYear

## Outputs
1. HeatingPowerDemand
1. TempRise
1. PoolReturnTemp(t+1h)
1. PoolReturnTemp(t+3h)

## Preprocessors
1. Extractor - accumulates rows of data. Data only reports on changes, so maintains history per row
python3 ./extractor.py > hpdata.csv  
1. Runtime filter - removes data when system is not operational. This is important becuase we can't measure pool temp then. Note: Removes first 5 minutes of each day as the system is stabilizing then.
python3 ./filterrunonly.py  hpdata.csv  > hpdatafiltered.csv
1. Time slicer - Averages data into 5 minute buckets


1. Future Capture - For each 5 minute bucket, looks forward 3 hours to determine pool temp rise. Special processing required for end of day processing. For last 3 hours, take temp rise * hours difference /3. Might be able to use that calculation for everything and just always look forward 36 buckets.


1. Convert for linux (if needed)
dos2unix hpdata.csv
1. Remove and NaN
sed -i '/nan/d' ./hpdata.csv # Remove nan entries

1. Drop last 3 hours of dataset as there is no forward looking data.


In [5]:
import pandas as pd
import numpy as np
from datetime import datetime


# Make numpy values easier to read.
np.set_printoptions(precision=3, suppress=True)

import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.layers.experimental import preprocessing

In [10]:

pool_train = pd.read_csv(
    "hpdatabuckets.csv",
    names={"last_changed":datetime,
           "hdpower":tf.float32,
           "pumppower":tf.float32,
           "poolheatedtemp":tf.float32,
           "pooltemp":tf.float32,
           "oa_temp":tf.float32,
           "uv_index":tf.float32,
           "efficiency":tf.float32,
           "fwd_min":tf.float32,
           "fwd_delta_temp":tf.float32,
           "fwd_delta_per_hr":tf.float32},
        
            low_memory=False)

pool_train.head()

,last_changed,hdpower,pumppower,poolheatedtemp,pooltemp,oa_temp,uv_index,efficiency,fwd_min,fwd_delta_temp,fwd_delta_per_hr
2020-06-22 16:30:11.285661-04:00,4742.0,835.0,94.0,90.0,90.4,59.0,5.0,8.435259384226065,174.813129,-1.0,-0.343224
2020-06-22 16:35:01.054978-04:00,0.0,293.0,90.0,90.0,90.4,59.0,5.0,inf,174.983538,-1.0,-0.342889
2020-06-22 16:40:00.076547-04:00,0.0,296.0,90.0,90.0,90.4,59.0,5.0,inf,175.032815,-1.0,-0.342793
2020-06-22 16:45:00.084615-04:00,0.0,297.0,90.0,90.0,90.4,59.0,5.0,inf,175.034219,-1.0,-0.342790
2020-06-22 16:50:01.165865-04:00,0.0,297.0,90.0,90.0,90.3,59.0,4.0,inf,174.981548,-1.0,-0.342893


In [25]:
pool_features = pool_train.copy()
pool_labels = pool_features.pop("hdpower")
pool_extran1 = pool_features.pop("last_changed")
pool_efficiency = pool_features.pop("efficiency")
pool_features_ary = np.array(pool_features)
pool_features_ary

array([[88. , 87. , 84.7,  9. ],
       [88. , 87. , 84.7,  9. ],
       [88. , 87. , 84.7,  9. ],
       ...,
       [70. , 67. , 71.5,  5. ],
       [70. , 67. , 71.5,  5. ],
       [70. , 67. , 71.5,  5. ]])

In [27]:
pool_model = tf.keras.Sequential([
  layers.Dense(64),
  layers.Dense(1)
])

pool_model.compile(loss = tf.losses.MeanSquaredError(),
                      optimizer = tf.optimizers.Adam())

In [28]:
pool_model.fit(pool_features_ary, pool_labels, epochs=10)


Epoch 1/10
31720/31720 [==============================] - 35s 1ms/step - loss: 127811.3672
Epoch 2/10
31720/31720 [==============================] - 40s 1ms/step - loss: 7850.0972
Epoch 3/10
31720/31720 [==============================] - 41s 1ms/step - loss: 7736.7720
Epoch 4/10
31720/31720 [==============================] - 39s 1ms/step - loss: 7688.4155
Epoch 5/10
31720/31720 [==============================] - 40s 1ms/step - loss: 7662.9077
Epoch 6/10
31720/31720 [==============================] - 42s 1ms/step - loss: 7652.5273
Epoch 7/10
31720/31720 [==============================] - 36s 1ms/step - loss: 7643.3999
Epoch 8/10
31720/31720 [==============================] - 38s 1ms/step - loss: 7644.9351
Epoch 9/10
31720/31720 [==============================] - 40s 1ms/step - loss: 7645.1372
Epoch 10/10
31720/31720 [==============================] - 41s 1ms/step - loss: 7638.9814


In [29]:
normalize = preprocessing.Normalization()


In [30]:
normalize.adapt(pool_features_ary)


In [31]:
norm_pool_model = tf.keras.Sequential([
  normalize,
  layers.Dense(64),
  layers.Dense(1)
])

norm_pool_model.compile(loss = tf.losses.MeanSquaredError(),
                           optimizer = tf.optimizers.Adam())

norm_pool_model.fit(pool_features_ary, pool_labels, epochs=10)

Epoch 1/10
31720/31720 [==============================] - 37s 1ms/step - loss: 2109081.7500
Epoch 2/10
31720/31720 [==============================] - 45s 1ms/step - loss: 7547.7812
Epoch 3/10
31720/31720 [==============================] - 41s 1ms/step - loss: 7549.1011
Epoch 4/10
31720/31720 [==============================] - 43s 1ms/step - loss: 7547.0664
Epoch 5/10
31720/31720 [==============================] - 42s 1ms/step - loss: 7549.0557
Epoch 6/10
31720/31720 [==============================] - 42s 1ms/step - loss: 7547.0718
Epoch 7/10
31720/31720 [==============================] - 42s 1ms/step - loss: 7548.0430
Epoch 8/10
31720/31720 [==============================] - 42s 1ms/step - loss: 7548.2134
Epoch 9/10
31720/31720 [==============================] - 40s 1ms/step - loss: 7548.0244
Epoch 10/10
31720/31720 [==============================] - 39s 1ms/step - loss: 7547.4258


In [32]:
dark_sky_temperture = np.array(pool_features['oa temp'])
dark_sky_temperture[:100]

array([84.7, 84.7, 84.7, 84.7, 84.7, 84.7, 84.7, 84.7, 84.7, 84.7, 84.7,
       84.7, 84.7, 84.7, 84.7, 84.7, 84.7, 84.7, 84.7, 84.7, 84.7, 84.7,
       84.7, 84.7, 84.7, 84.7, 84.7, 84.7, 84.7, 84.7, 84.7, 84.7, 84.7,
       84.7, 84.7, 84.7, 84.7, 84.7, 84.7, 84.7, 84.7, 84.7, 84.7, 84.7,
       84.7, 84.7, 84.7, 84.7, 84.7, 84.7, 84.7, 84.7, 84.7, 84.7, 84.7,
       84.7, 84.7, 84.7, 84.7, 84.7, 84.7, 84.7, 84.7, 84.7, 84.7, 84.7,
       84.7, 84.7, 84.7, 84.7, 84.7, 84.7, 84.7, 84.7, 84.7, 84.7, 84.7,
       84.7, 84.7, 84.7, 84.7, 84.7, 84.7, 84.7, 84.7, 84.7, 84.7, 84.7,
       84.7, 84.7, 84.7, 84.7, 84.7, 84.7, 84.7, 84.7, 84.7, 84.7, 84.7,
       84.7])

In [33]:
poolheatedtemp = np.array(pool_features['poolheatedtemp'])
poolheatedtemp[:10]

array([88., 88., 88., 88., 88., 88., 88., 88., 88., 89.])

In [34]:
poolreturntemp = np.array(pool_features['poolheatedtemp'])
poolreturntemp[:10]

array([88., 88., 88., 88., 88., 88., 88., 88., 88., 89.])

In [35]:
dp = np.array([[94, 90,90.4,5]])
print(dp)

hppower = pool_model.predict(dp)
print(hppower)


[[94.  90.  90.4  5. ]]
[[4773.806]]


In [36]:
dp = np.array([[93, 90,90.4,3]])
print(dp)

eff = pool_model.predict(dp)
print(eff)

[[93.  90.  90.4  3. ]]
[[4732.522]]


In [37]:
dp = np.array([[101, 95, 88,0]])
print(dp)

eff = pool_model.predict(dp)
print(eff)

[[101  95  88   0]]
[[5039.642]]


In [42]:
dp = np.array([[95,101, 88,10]])
print(dp)

eff = pool_model.predict(dp)
print(eff)

[[ 95 101  88  10]]
[[4807.699]]


In [38]:
pool_model.summary()

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_12 (Dense)             (None, 64)                320       
_________________________________________________________________
dense_13 (Dense)             (None, 1)                 65        
Total params: 385
Trainable params: 385
Non-trainable params: 0
_________________________________________________________________
